# Chapter 24: Class Metaprogramming

## Classes as Objects

## `type`: The Built-In Class Factory

In [5]:
class MySuperClass:
    pass

class MyMixin:
    pass

class MyClass(MySuperClass, MyMixin):
    x = 42
    
    def x2(self):
        return self.x * 2
    
obj = MyClass()
print(obj.x)
print(obj.x2())

42
84


In [4]:
MyClass2 = type('MyClass',
                (MySuperClass, MyMixin),
                {'x': 42, 'x2': lambda self:self.x*2},
                )

obj2 = MyClass2()
print(obj2.x)
print(obj2.x2())

42
84


In [ ]:
monkey = {'name': 'Monkey', 
        'attrs': ['age', 'weight','food']}

# How to create a Monkey class from this dict?
# attributes should not have default values and they'll
# be initialed in __init__ method.

# Use a class factory function.

## A Class Factory Function

In [10]:
# tag::RECORD_FACTORY[]
from typing import Union, Any
from collections.abc import Iterable, Iterator

FieldNames = Union[str, Iterable[str]]  # <1>

def record_factory(cls_name: str, field_names: FieldNames) -> type[tuple]:  # <2>

    slots = parse_identifiers(field_names)  # <3>

    def __init__(self, *args, **kwargs) -> None:  # <4>
        attrs = dict(zip(self.__slots__, args))
        attrs.update(kwargs)
        for name, value in attrs.items():
            setattr(self, name, value)

    def __iter__(self) -> Iterator[Any]:  # <5>
        for name in self.__slots__:
            yield getattr(self, name)

    def __repr__(self):  # <6>
        values = ', '.join(f'{name}={value!r}'
            for name, value in zip(self.__slots__, self))
        cls_name = self.__class__.__name__
        return f'{cls_name}({values})'

    cls_attrs = dict(  # <7>
        __slots__=slots,
        __init__=__init__,
        __iter__=__iter__,
        __repr__=__repr__,
    )

    return type(cls_name, (object,), cls_attrs)  # <8>


def parse_identifiers(names: FieldNames) -> tuple[str, ...]:
    if isinstance(names, str):
        names = names.replace(',', ' ').split()  # <9>
    if not all(s.isidentifier() for s in names):
        raise ValueError('names must all be valid identifiers')
    return tuple(names)
# end::RECORD_FACTORY[]

In [17]:
Dog = record_factory('Dog', 'name weight owner')
rex = Dog('Rex', 30, 'Bob')
rex

Dog(name='Rex', weight=30, owner='Bob')

In [18]:
monkey = {'name': 'Monkey', 
        'attrs': ['age', 'weight','food']}

Monkey = record_factory(monkey['name'], monkey['attrs'])
leo = Monkey(3, 30, 'banana')
print(leo)

Monkey(age=3, weight=30, food='banana')


## Introducing `__init__subclass__`